### Find Contours of Retinal Image

In [20]:
import cv2
import imutils
import numpy as np
import os
import matplotlib.pyplot as plt

#### Method #1 
* Method seems to result into disoriented retinal images

#### Remove Black Pixels on Background

In [21]:
train_path = "../../assets/datasets/odir5k/ODIR-5k/ODIR-5k/Training Images"

In [27]:
# abs_path = "preprocess_train" # not work cause already preprocessed
train_files = sorted(os.listdir(train_path))
rand_idx = np.random.randint(0, len(train_files))
image = cv2.imread(os.path.join(abs_path, train_files[rand_idx]))
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
print(image.shape)
# image = cv2.resize(image, None, fx=0.3, fy=0.3, interpolation=cv2.INTER_AREA)
cv2.imshow("Original", image)
cv2.waitKey(0)

def crop_image(image, plot=False, image_size=(256, 256), channel='y'):
    # mask of colored pixels
    mask = image > 0

    # coordinates of colored pixels
    coordinates = np.argwhere(mask)

    # binding box of non-black pixels.
    x0, y0, s0 = coordinates.min(axis=0)
    x1, y1, s1 = coordinates.max(axis=0) + 1 # slices are exclusive at the top

    # get the contents of the bounding box
    cropped = image[x0:x1, y0:y1]
    
    # convert to YUV for equalization
    img_yuv = cv2.cvtColor(cropped, cv2.COLOR_BGR2YUV)
    
    # apply adaptive equaliation on Y
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    if channel == 'y':
        img_yuv[:, :, 0] = clahe.apply(img_yuv[:, :, 0])
        
        # apply image normalization 
        mask = np.zeros((256, 256))
        normalized = cv2.normalize(img_yuv[:, :, 0], mask, 0, 255, cv2.NORM_MINMAX)
    
    elif channel == 'u':
        img_yuv[:, 0, :] = clahe.apply(img_yuv[:, 0, :])
        
        # apply image normalization 
        mask = np.zeros((256, 256))
        normalized = cv2.normalize(img_yuv[:, 0, :], mask, 0, 255, cv2.NORM_MINMAX)
    
    elif channel == 'v':
        img_yuv[0, :, :] = clahe.apply(img_yuv[0, :, :])
        
        # apply image normalization 
        mask = np.zeros((256, 256))
        normalized = cv2.normalize(img_yuv[0, :, :], mask, 0, 255, cv2.NORM_MINMAX)
    
    
    # apply image normalization 
#     mask = np.zeros((256, 256))
#     normalized = cv2.normalize(img_yuv[:, :, 0], mask, 0, 255, cv2.NORM_MINMAX)
    
    # convert back to bgr
    new_image = cv2.cvtColor(normalized, cv2.COLOR_YUV2BGR)
    new_image = cv2.resize(new_image, image_size, interpolation=cv2.INTER_AREA)
    
    if plot:
        plt.subplot(1,2,1)
        plt.imshow(image)
        plt.title(f"Original - {image.shape}")
        plt.axis('off')
        plt.grid(False)
        
        # ----------------- #
        
        plt.subplot(1,2,2)
        plt.imshow(new_image)
        plt.title(f"Preprocessed - {new_image.shape}")
        plt.axis('off')
        plt.grid(False)
        plt.show()
        
    return cropped

new_image1 = crop_image(image, plot=False, channel='y')
new_image2 = crop_image(image, plot=False, channel='u')
new_image3 = crop_image(image, plot=False, channel='v')
print(new_image1.shape)

cv2.imshow("CLAHE at Y", new_image1)
cv2.waitKey(0)

cv2.imshow("CLAHE at U", new_image2)
cv2.waitKey(0)

cv2.imshow("CLAHE at V", new_image3)
cv2.waitKey(0)

# overwrite the same file
# cv2.imshow("cropped", crop_image(image))
# cv2.waitKey(0)

cv2.destroyAllWindows()

(256, 256, 3)


error: OpenCV(4.5.1) c:\users\appveyor\appdata\local\temp\1\pip-req-build-kh7iq4w7\opencv\modules\imgproc\src\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,-1,-1>,struct cv::impl::A0x3a7f3a8f::Set<3,4,-1>,struct cv::impl::A0x3a7f3a8f::Set<0,2,5>,2>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1


In [26]:
rand_idx

5608

**Updated Preprocessing Methods**
* added adaptive histogram equalization
* added image normalization
* performs best

In [20]:
def keep_aspect_ratio(image):
    width_percentage = (256 / float(image.shape[0]))
    height_size = int((float(image.shape[1]) * (float(width_percentage))))
    cropped = cv2.resize(image, (256, height_size), interpolation=cv2.INTER_AREA)
    return cropped

image = cv2.imread("../../assets/images/3190_left.jpg")
image = cv2.resize(image, None, fx=0.3, fy=0.3 , interpolation=cv2.INTER_AREA)
cv2.imshow("Original", image)
cv2.waitKey(0)
# grayscale and blur to reduce noise
grayed = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
grayed = cv2.GaussianBlur(grayed, (5,5), 0)

# binarize image for erosion and dilation
# erosion removes the residual white noise and shrinks
# dilation enlarges the image since it has been noise-reduced
# threshold_image = cv2.threshold(grayed, 45, 255, cv2.THRESH_BINARY)[1]
threshold_image = cv2.erode(grayed, None, iterations=2)
threshold_image = cv2.dilate(threshold_image, None, iterations=2)

# find contours
contours = cv2.findContours(threshold_image.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)
c = max(contours, key=cv2.contourArea)

# extract bounding coords
extreme_pnts_left = tuple(c[c[:, :, 0].argmin()][0])
extreme_pnts_right = tuple(c[c[:, :, 0].argmax()][0])
extreme_pnts_top = tuple(c[c[:, :, 1].argmin()][0])
extreme_pnts_bot = tuple(c[c[:, :, 1].argmax()][0])

# crop and resize image to (256, 256)
new_image = grayed[extreme_pnts_top[1]:extreme_pnts_bot[1], extreme_pnts_left[0]:extreme_pnts_right[0]]
# new_image = cv2.equalizeHist(new_image) # boosted noise
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
equalized = clahe.apply(new_image)
# new_image = cv2.resize(new_image, (256, 256), interpolation = cv2.INTER_AREA)
normalizedImg = np.zeros((256, 256))
normalizedImg = cv2.normalize(equalized, normalizedImg, 0, 255, cv2.NORM_MINMAX)
cropped = cv2.resize(normalizedImg, (256, 256), interpolation=cv2.INTER_AREA)
cropped = cv2.cvtColor(cropped, cv2.COLOR_GRAY2BGR)
print(normalizedImg.shape)
# keep aspect ratio
# cropped = keep_aspect_ratio(normalizedImg)
print(cropped.shape)
cv2.imshow("Cropped without Adaptive HE", new_image)
cv2.waitKey(0)
cv2.imshow("Cropped with Adaptive HE", equalized)
cv2.waitKey(0)
cv2.imshow("Equalized and Normalized", cropped)
cv2.waitKey(0)

cv2.destroyAllWindows()



(369, 486)
(337, 256)


#### Method #2
* Performs better than prior method but still results in fewer disoriented images

In [24]:
image = cv2.imread("../../assets/images/3080_right.jpg")
image = cv2.resize(image, None, fx=0.3, fy=0.3, interpolation=cv2.INTER_AREA)
test = image.copy()

cv2.imshow("Original", test)
cv2.waitKey(0)

# gray and gaussian blur to reduce salt and pepper
test = cv2.cvtColor(test, cv2.COLOR_BGR2GRAY)
test = cv2.GaussianBlur(test, (5,5), 0)

# erode and dilate 
eroded = cv2.erode(test, None, iterations=2)
eroded = cv2.dilate(test, None, iterations=2)

cv2.imshow("Eroded and Dilated", eroded)
cv2.waitKey(0)

# Find the Canny edges
# to better find the external contours of the image
edges = cv2.Canny(eroded, 20, 70)

# denoise the image
edges = cv2.fastNlMeansDenoising(edges, None, 7, 7, 21)
cv2.imshow("Canny Edges", edges)
cv2.waitKey(0)

# find the contours
contours = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
print("COntours Found: ", len(contours))
cv2.imshow("Canny Edges After Contouring", edges)
cv2.waitKey(0)
contours = imutils.grab_contours(contours)

# draw contours
# -1 draws all contours
# 0 draws first 
# 1 draws second
cv2.drawContours(image, contours, -1, (0, 255, 0), 2)
# intersection = cv2.bitwise_and(image, eroded)
cv2.imshow("Contours", image)
cv2.waitKey(0)
# extract bouding coords
c = max(contours, key=cv2.contourArea)
extreme_pnts_left = tuple(c[c[:, :, 0].argmin()][0])
extreme_pnts_right = tuple(c[c[:, :, 0].argmax()][0])
extreme_pnts_top = tuple(c[c[:, :, 1].argmin()][0])
extreme_pnts_bot = tuple(c[c[:, :, 1].argmax()][0])

new_image = test[extreme_pnts_top[1]:extreme_pnts_bot[1], extreme_pnts_left[0]:extreme_pnts_right[0]]
new_image = cv2.resize(new_image, (256, 256), interpolation=cv2.INTER_AREA)

cv2.imshow("Cropped", new_image)
cv2.waitKey(0)

cv2.destroyAllWindows()

COntours Found:  2


In [10]:
import numpy as np

image = cv2.imread("../../assets/images/3190_left.jpg")
image = cv2.resize(image, None, fx=0.3, fy=0.3, interpolation=cv2.INTER_AREA)

# red color boundaries [B, G, R]
lower = [1, 0, 20]
upper = [60, 40, 220]

# create numpy arrays from the boundaries
lower = np.array(lower, dtype='uint8')
upper = np.array(upper, dtype='uint8')

# find the colors within the specified boundaries and apply the mask
mask = cv2.inRange(image, lower, upper)
output = cv2.bitwise_and(image, image, mask=mask)

ret, thresh = cv2.threshold(mask, 40, 255, cv2.THRESH_BINARY)
# if (cv2.__version__[0] > 3.0):
#     contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# else:
#     im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)    

if len(contours) != 0:
    # draw in blue the contours that were found
    cv2.drawContours(output, contours, -1, (255, 0, 0), 3)
    
    # find the largest contour (c) by the area
    c = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(c)
    
    # draw the largest contour (c) in green
    cv2.rectangle(output, (x,y), (x+w, x+h), (0, 255, 0), 2)
    
# show the images
cv2.imshow("Result", np.hstack([image, output]))
cv2.waitKey(0)

new_image = image[y:y+h, x:x+w]
new_image = cv2.resize(new_image, (256, 256), interpolation=cv2.INTER_AREA)
cv2.imshow("Cropped", new_image)
cv2.waitKey(0)
cv2.destroyAllWindows()